In [1]:
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle as pkl
import imblearn
np.random.seed(1332)
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys                                               #### Filling the missing values using KNN because the dataset is very less
from impyute.imputation.cs import fast_knn
import imblearn                                      ### sampling the data using the smote method
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import sklearn
import xgboost
from xgboost import XGBClassifier              ### XGBoost classification method
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import cv


In [2]:
excel = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
df = pd.read_excel(excel, 'BL+3M')

In [ ]:
df

In [3]:
df = df.drop(columns = ['SubjectID', 'CDate', 'Match'])    ### Unique values
df = df.drop(columns = ['SubjectID.1', 'UNMC_id.1', 'CDate.1', 'init_group.1', 'grp.1', 'UNMC_id.2', 'grp.2', 'init_group.2', 'CDate.1', 'futime.1'])    ### Duplicate values
df = df.drop(columns = ['init_group', 'futime'])    ### same values for all rows

In [ ]:
df.isnull().mean()[df.isnull().mean() > 0.7]   

In [4]:
df = df.drop(columns = ['statin_use', 'rfstatus_impute', 'ccpstatus_impute', 'statin_use.1', 'smkyrs', 'numcigs'])  ### Dropping columns who have null values greater than 70%

In [5]:
df = df.drop(columns = ['DAS28-CRP 3m', 'DAS28-CRP BL'])

In [6]:
df = df.rename(columns={"usresultsIgA.1": "usresultsIgA_BL", "usresultsIgG.1": "usresultsIgG_BL", "usresultsIgM.1": "usresultsIgM_BL", 'seatedbp1.1': 'seatedbp1_BL',
                              'seatedbp2.1': 'seatedbp2_BL', 'pres_mtx.1': 'pres_mtx_BL', 'pres_arava.1': 'pres_arava_BL', 'pres_azulfidine.1': 'pres_azulfidine_BL', 
                              'pres_plaquenil.1': 'pres_plaquenil_BL', 'pres_imuran.1': 'pres_imuran_BL', 'pres_minocin.1': 'pres_minocin_BL', 'pres_pred.1': 'pres_pred_BL',
                              'statin_use.1': 'statin_use_BL', 'tender_jts_28.1': 'tender_jts_28_BL', 'BLswollen_jts_28': 'swollen_jts_28_BL',
                              'BLmd_global_assess': 'md_global_assess_BL', 'BLpt_global_assess': 'pt_global_assess_BL', 'BLdi': 'di_BL', 'BLpt_pain': 'pt_pain_BL', 'BLusresultsCRP': 'usresultsCRP_BL',
                              'DAS28-CRP BL': 'DAS28-CRP_BL', })     ### Renaming some columns. Not sure why - did it because previous student did it

In [7]:
df = df.drop('UNMC_id',1)
final_df = df
model_label = LabelEncoder()
final_df['grp'] = model_label.fit_transform(final_df['grp'].astype('str'))
final_df['gender'] = model_label.fit_transform(final_df['gender'].astype('str'))
final_df['final_education'] = model_label.fit_transform(final_df['final_education'].astype('str'))
final_df['race_grp'] = model_label.fit_transform(final_df['race_grp'].astype('str'))
final_df['newsmoker'] = model_label.fit_transform(final_df['newsmoker'].astype('str'))
final_df['drinker'] = model_label.fit_transform(final_df['drinker'].astype('str'))
final_df['ara_func_class'] = model_label.fit_transform(final_df['ara_func_class'].astype('str'))

/state/partition1/job-14953214/ipykernel_11681/2061900728.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('UNMC_id',1)


In [8]:
final_df = final_df[final_df['3MResponse'] != 'Unknown']### We don't need unknown category in our classification data so removing that category



In [ ]:
# from scipy import stats
# def z_score(data):
#     data_new = data.drop('3MResponse',1)
#     z_scores = stats.zscore(data_new)
#     abs_z_scores = np.abs(z_scores)
#     print(data_new)
#     filtered_entries = (abs_z_scores < 3).all(axis=1)
#     print(filtered_entries.value_counts())
#     new_df = data_new[filtered_entries]
#     print(new_df.shape)
#     return new_df
# X = z_score(final_df)
# y = final_df.iloc[X.index,-1]
# y = model_label.fit_transform(y.astype('str'))

In [9]:
X = final_df.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = final_df['3MResponse']
#y = model_label.fit_transform(y.astype('str'))

/state/partition1/job-14953214/ipykernel_11681/42841613.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = final_df.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [ ]:
y

In [10]:
def KNN(new_df):
    sys.setrecursionlimit(100000) 
    imputed_training=fast_knn(new_df.values, k=15)
    return imputed_training
imputed_training = KNN(X)
c = list(X.columns)
df_without_nulls = pd.DataFrame(imputed_training, columns=c)

In [11]:
lab = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']

In [12]:
from scipy import stats
for i in df_without_nulls.columns:
    if i not in lab:
        IQR = df_without_nulls[i].quantile(0.75) - df_without_nulls[i].quantile(0.25)
        lower_limit = df_without_nulls[i].quantile(0.25) - 1.5 * IQR
        upr_limit = df_without_nulls[i].quantile(0.75) + 1.5 * IQR
        upr = df_without_nulls[i] > upr_limit
        low = df_without_nulls[i] < lower_limit
        #df_without_nulls = df_without_nulls[(df_without_nulls[i] < upr_limit) and (df_without_nulls[i] > lower_limit)].all()
        df_without_nulls = df_without_nulls[~low | upr]
        
print(df_without_nulls.shape)     
        
# def z_score(data_new):
#     z_scores = stats.zscore(data_new)
#     abs_z_scores = np.abs(z_scores)
#     print(data_new)
#     filtered_entries = (abs_z_scores < 3).all(axis=1)
#     print(filtered_entries.value_counts())
#     new_df = data_new[filtered_entries]
#     print(new_df.shape)
#     return new_df
# X_1 = z_score(df_without_nulls)


(155, 77)


In [ ]:
# for i in df_without_nulls.columns:
#     if df_without_nulls[i].isnull().sum() == 155:
#         print(i)

In [ ]:
print(type(y))

In [ ]:
df_without_nulls.index

In [ ]:
y_1 = final_df.iloc[df_without_nulls.index,0]

In [ ]:
y_1

In [13]:
#y = 
y= final_df.iloc[df_without_nulls.index,0]
y = model_label.fit_transform(y.astype('str'))

#print(y.shape, y)

In [14]:
y

array([1, 0, 0, 1, 0, 0, 2, 0, 2, 0, 1, 0, 2, 1, 1, 2, 0, 2, 1, 0, 1, 2,
       1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 2, 2, 0,
       1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 1, 0, 2, 1, 0, 2, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 2, 2, 1, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 1, 1, 1,
       2, 1, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 2, 2, 1, 2, 2, 0, 1, 2, 0, 1,
       2, 1, 0, 0, 1, 0, 0, 1, 0, 2, 1, 1, 2, 1, 0, 0, 1, 0, 2, 1, 1, 2,
       0])

In [15]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in df_without_nulls.columns:
    if i not in labels:
        mean = df_without_nulls[i].mean()
        std = df_without_nulls[i].std()
        df_without_nulls[i] = (df_without_nulls[i] - mean) / std

In [16]:
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    df_without_nulls = df_without_nulls.drop(i,1)

/state/partition1/job-14953214/ipykernel_11681/1131187374.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_without_nulls = df_without_nulls.drop(i,1)


In [74]:
rs = 100

In [75]:
X_train_norm, X_val_norm, Y_train, Y_val = train_test_split(df_without_nulls, y, random_state=rs, test_size=0.1)     ### splitting the dataset into training and test set

In [76]:
def sampling(over_under,X,y):
    if over_under == 'under':
        sampler = imblearn.under_sampling.RandomUnderSampler(random_state = rs)
        X_under, y_under = sampler.fit_resample(X, y)
        return X_under,y_under
    elif over_under == 'over':
        ros = RandomOverSampler(random_state=rs)
        X_over, y_over = ros.fit_resample(X, y)
        return X_over,y_over
    else:
        sampler = SMOTE()
        X_smote, y_smote = sampler.fit_resample(X, y)
        return X_smote,y_smote
#print(X)
#print(y)
X_final,y_final = sampling('smote',X_train_norm,Y_train)
#X_final,y_final = X_train_norm, Y_train

arr = [0.8214, 0.5714, 0.7142, 0.5714, 0.75, 0.6428, 0.8214, 0.7857, 0.5357,0.75]
arr1 = [0.7195, 0.7378, 0.7440, 0.7369, 0.7104, 0.7250, 0.7163, 0.7063, 0.7623, 0.7487]
print(np.std(arr), np.std(arr1))

In [77]:
model3 = None
#print(model3)
def random_forest(X_train_norm, X_test, Y_train,Y_test):
    global model3
    #print(model3)
    model3 = sklearn.ensemble.RandomForestClassifier(n_estimators=474, oob_score = True, criterion='gini',max_depth=90, min_samples_split=10, random_state=rs, max_features='auto', min_samples_leaf = 6)
    model3.fit(X_train_norm, Y_train)
    print(model3.oob_score_)
    print(f"Training accuracy is {model3.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model3.score(X_test, Y_test)}")
    #sklearn.metrics.plot_confusion_matrix(model3, X_test, Y_test)
    #sklearn.metrics.plot_roc_curve(model3, X_test, Y_test)
random_forest(X_final, X_val_norm, y_final, Y_val)

0.6839080459770115
Training accuracy is 0.9655172413793104
Testing accuracy is 0.8125


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [57]:
from sklearn import svm
from sklearn.model_selection import KFold
model_svm = None
def svm1(X_train_norm, X_test, Y_train,Y_test):
    global model_svm
    model_svm = svm.SVC(random_state = rs, kernel = 'poly')
    model_svm.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model_svm, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results.mean())
    print(f"Training accuracy is {model_svm.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model_svm.score(X_test, Y_test)}")
    
svm1(X_final, X_val_norm, y_final, Y_val)
    

sklearn::: 0.7415032679738561
Training accuracy is 0.9367816091954023
Testing accuracy is 0.3125


In [37]:
from mlxtend.classifier import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
lr = sklearn.linear_model.LogisticRegression()  # defining meta-classifier
KNC = XGBClassifier()   # initialising KNeighbors Classifier
NB = sklearn.ensemble.RandomForestClassifier()
clf_stack = StackingClassifier(classifiers =[KNC, NB], meta_classifier = lr, use_probas = True, use_features_in_secondary = True)
clf_stack = clf_stack.fit(X_final, y_final)
kfold = StratifiedKFold(n_splits=10)
results = cross_val_score(clf_stack, X_final, y_final, cv=10)
print("sklearn:::", results.mean())
print(f"Training accuracy is {clf_stack.score(X_final, y_final)}")
print(f"Testing accuracy is {clf_stack.score(X_val_norm, Y_val)}")

/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:56:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:56:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:56:54] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/ext3/miniconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:56:54] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
sklearn::: 0.7758169934640523
Training accuracy is 1.0
Testing accuracy is 0.5


/ext3/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
model5 = None
def logistic_regress(X_train_norm, X_test, Y_train,Y_test):
    global model5
    model5 = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='saga', penalty = 'elasticnet', C = 1, l1_ratio = 0.5, random_state=rs)
    model5.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model5, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results.mean())
    print(f"Training accuracy is {model5.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model5.score(X_test, Y_test)}")
    #sklearn.metrics.plot_confusion_matrix(model3, X_test, Y_test)
    #sklearn.metrics.plot_roc_curve(model3, X_test, Y_test)
logistic_regress(X_final, X_val_norm, y_final, Y_val)

In [ ]:
#data_dmatrix = xgboost.DMatrix(data=X_final,label=y_final)
model4 = None
def xgboost(X_train_norm, X_test,Y_train, Y_test):
    global model4
    #model4 = XGBClassifier(n_estimators = 300, random_state=rs, max_depth=100, eta=0.5, gamma=15, min_child_weight=4, alpha=44, reg_lambda=0.44, colsample_bytree = 0.5, sampling_method = 'uniform')
    model4 = XGBClassifier(n_estimators = 200, random_state=rs, subsample=0.8, max_depth=50, eta=0.5, gamma=1, min_child_weight=10, alpha=1)

    model4.fit(X_train_norm, Y_train)
    kfold = StratifiedKFold(n_splits=10)
    results = cross_val_score(model4, X_train_norm, Y_train, cv=10)
    print("sklearn:::", results, results.mean())
    print(f"Training accuracy is {model4.score(X_train_norm, Y_train)}")
    print(f"Testing accuracy is {model4.score(X_test, Y_test)}")
xgboost(X_final, X_val_norm, y_final, Y_val)

In [ ]:
x = (0,1,2)
y = (70.99, 67.85, 70.01)
plt.figure()
yerr = np.array([(2.81,1.5),(3.6, 3.7), (2.6, 3.1)]).T
plt.errorbar(x, y, yerr, fmt='r^')
plt.xlabel("Algorithms")
plt.ylabel("Accuracy in %")
plt.title("validation/oob score")
plt.text(3.5, 70, '0 = Logistric Regression', fontsize = 23)
plt.text(3.5, 69, '1 = Random Forest', fontsize = 23)
plt.text(3.5, 68, '2 = XGBoost', fontsize = 23)
plt.xticks([0,1,2])

#plt.legend(['First line', 'Second line','Third Line'])
plt.show()

x_test = [0, 1, 2]
y_test = [66.42, 64.63, 64.63]
plt.figure()
y_err_test = np.array([(16.42, 12.14),(21.78, 13.93), (21.78, 13.93)]).T
plt.errorbar(x_test, y_test, y_err_test, fmt='r^')
plt.title("test score")
plt.xlabel("Algorithms")
plt.ylabel("Accuracy in %")
plt.text(3.5, 70, '0 = Logistric Regression', fontsize = 23)
plt.text(3.5, 65, '1 = Random Forest', fontsize = 23)
plt.text(3.5, 60, '2 = XGBoost', fontsize = 23)
plt.xticks([0,1,2])

plt.show()